# Informe Final Carrera
Profundiza y ajusta la resolución de tus proyectos: agrega al menos una fuente de datos o prueba un modelo que hasta el momento no hayas aplicado. Elabora un informe final que describa el proceso de toma de decisiones. Sustenta la razón por la cual usaste las librerías y aplicaste los métodos al dataset. Todos tus hallazgos deberán ser comunicados en función de la problemática y las preguntas que buscabas responder.

## Introducción

En el presente Proyecto, se trabajará con un **nuevo caso de estudio**, con una **nueva problemática**, no relacionada con los Proyectos anteriores presentados, por lo que en el repositorio de GitHub podrán encontrar el presente Notebook y el Archivo csv con el que se trabajará.

## Fraude Bancario

Se abordará la problemática del **Fraude Bancario**, el cual es un negocio de miles de millones de dólares y está aumentando cada año.  
A saber, una organización típica pierde aproximadamente el 5% de sus ingresos anuales debido al fraude.  
En un contexto de cambiante panorama bancario global, donde las redes de sucursales se están reduciendo, los volúmenes de pagos digitales aumentan y los pagos se procesan en segundos, los estafadores están encontrando creativamente nuevas formas de robar a los bancos y sus clientes.

De acuerdo a la [Encuesta Global de Fraude Bancario 2019](https://home.kpmg/cr/es/home/tendencias/2019/07/fraude_bancario.html) realizado por KPMG [1], podemos observar, entre los hallazgos principales, los siguientes:
* Más de la mitad de los encuestados en todo el mundo experimentaron aumentos tanto en el valor total como en el volumen del fraude externo. El aumento de las tipologías de fraude en todo el mundo desde 2015 hasta 2018 incluye el robo de identidad y la toma de posesión de la cuenta, los ataques cibernéticos, las tarjetas sin fraude y las estafas autorizadas de pago automático;
* Los bancos a nivel mundial están viendo una tendencia creciente en las estafas. Los estafadores están manipulando y obligando a los clientes a hacer pagos a ellos, sin pasar por los controles bancarios;
* La Banca Abierta se considera un desafío importante en el riesgo de fraude de los bancos, ya que los bancos de todo el mundo se están preparando para abrir sus puertas a terceros para acceder a los datos de sus clientes;
* Los estafadores son cada vez más sofisticados y pueden cambiar y adaptar rápidamente sus enfoques. Los bancos deben ser ágiles para responder a las nuevas amenazas y adoptar nuevos enfoques y tecnologías para predecir y prevenir el fraude;
* En cada región, los bancos encuestados consideraron que el desafío más importante en el riesgo de fraude son los ciberataques.

## ¿Por qué aplicar modelos ML para detectar Fraude Bancario?

Los algoritmos de **Machine Learning** (aprendizaje automático) procesan los datos en bruto (sin procesar), como correos electrónicos o mensajes de texto, y luego aprenden de lo que toman como entrada, volviéndose más inteligentes a lo largo del camino.  
Por otro lado, los **métodos basados en Reglas** no pueden detectar ningún patrón nuevo en los datos, ya que sólo siguen un escenario preestablecido, que no incluye patrones de actividad fraudulenta ligeramente modificados.

## BankSim Dataset

* A partir de ahora, se buscará detectar las transacciones fraudulentas del conjunto de datos de **BankSim**.  
* BankSim es un **simulador de pagos bancarios** basado en agentes, y en una muestra de datos transaccionales agregados, proporcionados por un banco en España.
* El objetivo principal de BankSim es la **generación de datos sintéticos** que se pueden utilizar para la investigación de detección de fraudes.
* Consiste en pagos de varios clientes hechos en diferentes períodos de tiempo y con diferentes cantidades.  
* Para mayor información sobre el conjunto de datos (originalmente nombrado 'bs140513_032310.csv'), se puede consultar la página de [Kaggle](https://www.kaggle.com/ntnu-testimon/banksim1) [2].  

Los pasos a seguir serán los siguientes:

1. [Análisis Exploratorio de Datos (EDA)](#1.-Análisis-Exploratorio-de-Datos)
2. [Preprocesamiento de Datos](#2.-Preprocesamiento-de-Datos)
3. [PCA (Principal Component Analysis)](#3.-PCA-(Principal-Component-Analysis))
4. [Sobremuestreo (Oversampling) con SMOTE](#4.-Sobremuestreo-(Oversampling)-con-SMOTE)
5. [Modelos de Machine Learning](#5.-Modelos-de-Machine-Learning)
6. [Comparación del Desempeño de los Modelos Desarrollados](#6.-Comparación-del-Desempeño-de-los-Modelos-Desarrollados)
7. [Conclusión](#7.-Conclusión)
8. [Recursos](#8.-Recursos)

### 1. Análisis Exploratorio de Datos

1. __Se importan las librerías__ necesarias para trabajar en la consigna.

In [ ]:
import numpy as np # Para hacer cálculos numéricos con Python usando arrays
import pandas as pd # Para manipulación y análisis de datos con Python (por Ej. tablas numéricas)
import matplotlib.pyplot as plt # Herramienta de visualización
import seaborn as sns # Herramienta de visualización
sns.set() # interfaz preferida

2. __Se realiza la carga el dataset__ usando las funcionalidades de Pandas.

In [ ]:
banksim = pd.read_csv('banksim.csv')

In [ ]:
banksim.shape

* *El Dataset, cuenta con **594.643 Filas**, y **10 Columnas**.*

In [ ]:
banksim.head()

3. Se realiza una __descripción__ de las características:  

* *step:* representa el día desde el comienzo de la simulación. Tiene 180 pasos, por lo que la simulación dura virtualmente 6 meses. 
* *customer:* la identificación del cliente.  
* *age:* edad categorizada, las cuales se detallan a continuación:  
    * 0: <= 18,  
    * 1: 19-25,  
    * 2: 26-35,  
    * 3: 36-45,  
    * 4: 46-55,  
    * 5: 56-65,  
    * 6: > 65  
    * U: Desconocido  
* *gender:* género del cliente.  
    * E: Enterprise,  
    * F: Mujer,  
    * M: Hombre,  
    * U: Desconocido  
* *zipcodeOri:* el código postal de origen/fuente.  
* *merchant:* la identificación del comerciante.  
* *zipMerchant:* el código postal del comerciante.  
* *category:* categoría de la compra. A posteriori se verán con más detalle.  
* *amount:* monto (gasto) que implica la compra.  
* *fraud:* variable objetivo que muestra si la transacción es fraudulenta **(1)** o no **(0)**.

4. __Info por Columna:__ Se imprimen los nombres de cada columna, se observan sus valores faltantes y se identifica a qué tipo de dato corresponden.

In [ ]:
banksim.info()

* *Ninguna columna tiene valores faltantes, lo cual es lógico por tratarse de un dataset sintético.*
* *Por ello, es que no será necesario realizar imputación de valores.*

5. Análisis del feature **Fraude**.

In [ ]:
pd.unique(banksim['fraud'])

In [ ]:
ax=sns.countplot(data = banksim, x = 'fraud', order = banksim['fraud'].value_counts().index, palette='pastel')
ax.set_xticklabels(["No Fraudulentas","Faudulentas"])
plt.xlabel('Tipo de Transacción');
plt.ylabel('Cantidad de Transacciones');
plt.title('Cantidad de Transacciones por Tipo')

* *Como se observa en el gráfico anterior, los datos de las transacciones en fraudulentas y no, según el recuento de instancias, se encuentran **desequilibrados**, siendo las transacciones no fraudulentas casi imperceptibles a la vista*.
* *Para equilibrar el conjunto de datos se pueden realizar **técnicas de sobremuestreo o submuestreo**.*  
* *El **sobremuestreo** es aumentar el número de la clase minoritaria, generando nuevas instancias de dicha clase.*  
* *El **submuestreo** consiste en reducir el número de instancias de la clase mayoritaria, mediante la selección de puntos aleatorios de la misma hasta que sea igual a la clase minoritaria.*  
* *Ambas operaciones tienen algunos **riesgos**: El sobremuestreo creará copias o puntos de datos similares que a veces no serían útiles para el caso de la detección de fraudes porque las transacciones fraudulentas pueden variar. El submuestreo significa que perdimos puntos de datos y, por lo tanto, información.*

In [ ]:
print(banksim['fraud'].value_counts().sum())

In [ ]:
print(banksim['fraud'].value_counts())

* *Vemos que las instancias no fraudulentas representan un 98,8% del total, pudiendo corroborarse el desbalanceo del dataset.*

6. Análisis del **Fraude por Categoría**.

In [ ]:
print(banksim['category'].value_counts())

* *Se cuenta con 15 tipos de categoría diferentes, siendo la más recurrente `es_transportation`, abarca el 85% de las transacciones.*

7. Análisis del **Fraude por Monto o Gasto**.

* *Confirmemos lo expuesto en el punto anterior, comprobando la cantidad de fraude y de no fraude que se ha tramitado.*

In [ ]:
corr = banksim[['fraud','amount']].corr()
plt.figure(figsize=(6,6))
sns.heatmap(corr, cbar = True,  square = True, annot=True, fmt= '.2f',annot_kws={'size': 15}, cmap='Pastel2')
plt.xticks(rotation = 45)
plt.yticks(rotation = 45)
plt.title('Correlación entre Variables - Pearson')
plt.show()

* *Existe una correlación positiva entre fraude y monto o gasto insumido en la trasacción, aunque no muy elevada.*

* *A continuación, se exponen el gasto promedio y el porcentaje de fraude por categoría.*

In [ ]:
print("Gasto Promedio y Porcentaje de Fraude por Categoría",banksim.groupby('category')['amount','fraud'].mean())

* *Parece que el ocio (`es_leisure`) y los viajes (`es_travel`) son las categorías más seleccionadas por los defraudadores.*
* *Los defraudadores eligieron las categorías en las que la gente **gasta más en promedio**.*

* *Para confirmar lo antes expuesto, se crean dos Dataframes diferenciando las transacciones fraudulentas de las que no lo son.*

In [ ]:
fraudulentas = banksim.loc[banksim.fraud == 1] 
no_fraudulentas = banksim.loc[banksim.fraud == 0]

In [ ]:
pd.concat([fraudulentas.groupby('category')['amount'].mean(),no_fraudulentas.groupby('category')['amount'].mean(),\
           banksim.groupby('category')['fraud'].mean()*100],keys=["Fraudulentas","No Fraudulentas","Porcentaje(%)"],axis=1,\
          sort=False).sort_values(by=['Porcentaje(%)'],ascending=False)

* *De acuerdo a lo observado anteriormente, podemos decir que para un determinado tipo de categoría, una transacción fraudulenta tendrá un gasto (`amount`) mucho mayor -unos cuatro veces o más-, que el gasto **promedio** para dicha categoría.*

* *Se trazan los box plots de los gastos realizados -de transacciones fraudulentas y no fraudulentas- por tipo de categoría.* 

In [ ]:
plt.figure(figsize=(15,15))
sns.boxplot(data = banksim, x = 'amount', y = 'category', palette='pastel')
plt.title("Gasto Realizado por Tipo de Categoría", fontsize=12)
plt.xlabel("Gasto",fontsize=12)
plt.ylabel("Categoría",fontsize=12)
plt.legend()
plt.show()

* *Las categorías tienen un gasto promedio similar, **entre 0 y 500** o **600** quizás (descartando los valores atípicos muy dispersos).*
* *La excepción observable, es la categoría de **viajes**, cuyo gasto se encuentra **muy por encima** de las demás categorías.*

In [ ]:
plt.hist(data = fraudulentas, x = 'amount', alpha=0.5, label='Fraudulentas',bins=100)
plt.hist(data = no_fraudulentas, x = 'amount', alpha=0.5, label='No Fraudulentas',bins=100)
plt.title("Histograma de Transacciones Fraudulentas y No Fraudulentas")
plt.xlabel('Monto de la Transacción');
plt.ylabel('Cantidad de Transacciones');
plt.ylim(0,10000)
plt.xlim(0,1000)
plt.legend()
plt.show()

* *Una vez más podemos ver en el histograma, que las transacciones fradulentas son menos en cantidad pero más en monto.*
* *Con montos mayores a 400, ya poderíamos sospechar que hay fraude.*

8. Análisis del **Fraude por Edad**.

In [ ]:
print(banksim['age'].value_counts())

* *Como vimos anteriormente, las transacciones se encuentran categorizadas por franjas etarias, y la categoría 'U' representa las edades desconocidas.*
* *La franjas que realizaron más transacciones, son las 2, 3 y 4 (en ese órden), es decir, aquellos que tienen entre 26 a 55 años, lo cual es lógico, ya que se trata de personas en edad laboral y que además se han adaptado al uso de herramientas tecnológicas.*

In [ ]:
print((banksim.groupby('age')['fraud'].mean()*100).reset_index().rename(columns={'age':'Age','fraud' : 'Fraud Percent'}).sort_values(by='Fraud Percent'))

* *El fraude **ocurre más en edades iguales y menores de 18 años (0º categoría)**.*
* *Puede ser que los estafadores realmente sean jóvenes menores de 18 años, aunque lo más probable es que aquellos que comenten fraude, piensen que habría menos consecuencias si muestran su edad más joven.*

9. Análisis del feature **Pasos**.

In [ ]:
print(banksim['step'].value_counts())

* *Como se indicó anteriormente, `step` representa el día desde el comienzo de la simulación, abarcando 180 días (o 180 pasos).*
* *Por lo visto, la distribución de los datos por cada día parece bastante equilibrado, representando el día con más transacciones (día 175) un 0,6% del total, y el de menos operaciones (día 1) un 0,4%, y mostrando una evolución del número de transacciones del 0,03% mensual.*

10. Análisis del feature **Cliente**.

In [ ]:
print(banksim['customer'].value_counts())

* *Tenemos un total de 4.112 clientes.*
* *El cliente que mayor cantidad de transacciones realizó, es 'C1978250683', con 265 operaciones (lo cual representa un 0,04% de total de transacciones).*
* *`customer` no parece ser una variable determinante a la hora de establecer si una transacción es fraudulenta o no.*

11. Análisis de la variable **Género**.

In [ ]:
print(banksim['gender'].value_counts())

* *Los géneros Femenino y Masculino son los que más transacciones realizaron, es decir, que son personas físicas las que más operaciones efectuaron, mientras que son pocas las Empresas y mucho menos los géneros desconocidos.*

12. Análisis de los **Zip Codes**.

In [ ]:
print(banksim['zipcodeOri'].value_counts())

In [ ]:
print(banksim['zipMerchant'].value_counts())

* *Como se observa, hay un sólo tipo de Zip Code, tanto de Origen como de Comerciante, por lo que luego, en el preprocesamiento de datos, los descartaremos.*

13. Análisis del feature **Comerciante**.

In [ ]:
pd.unique(banksim['merchant'])

In [ ]:
print(banksim['merchant'].value_counts())

* *Tenemos un total de 50 comerciantes.*
* *El comerciante que mayor cantidad de transacciones recibió, es 'M1823072687', con 299.693 operaciones (50,4% de total de transacciones) y el segundo es 'M348934600', con 205426 operaciones (34,5%).*
* *Entre el primer y segundo comerciante con más operaciones, representan el 84,9% del total de transacciones.*

In [ ]:
pd.concat([fraudulentas.groupby('merchant')['amount'].mean(),no_fraudulentas.groupby('merchant')['amount'].mean(),\
           banksim.groupby('merchant')['fraud'].mean()*100],keys=["Fraudulentas","No Fraudulentas","Porcentaje(%)"],axis=1,\
          sort=False).sort_values(by=['Porcentaje(%)'],ascending=False)

* *Parece ser importante el tipo de comerciante a la hora de establecer si una transacción es o no fraudulenta, de acuerdo al gasto promedio expuesto para cada uno de ellos.*
* *Un comerciante que presenta un elevado monto promedio en sus operaciones, podría ser, por ejemplo, una determinada agencia de viajes.*

### 2. Preprocesamiento de Datos

1. Descartamos las columnas de **Zip Code** y **Customer**.

* *Las 3 variables eliminadas, se consideran poco relevantes para nuestro análisis, de acuerdo a lo expuesto en los puntos anteriores.*

In [ ]:
banksim_filtrado = banksim.drop(['customer','zipcodeOri','zipMerchant'],axis=1)

In [ ]:
banksim_filtrado.head()

* *Vemos que se eliminaron ambas columnas de Zip Codes y Customer.*

In [ ]:
banksim_filtrado.dtypes

2. __Encoders:__ Se realizará la transformación de los datos.

* *Los features categóricos serán transformados a valores numéricos.* 
* *Las categorías a trabajar, son nominales, es decir que no tienen un orden.*
* *Lo ideal sería realizar dummies, pero por el tamaño del dataset (más de 500.000 instancias) podría demorarse demasiado.*

In [ ]:
for col_name in banksim_filtrado.columns:
    if(banksim_filtrado[col_name].dtype == 'object'):
        banksim_filtrado[col_name]= banksim_filtrado[col_name].astype('category')
        banksim_filtrado[col_name] = banksim_filtrado[col_name].cat.codes

In [ ]:
banksim_filtrado.head()

In [ ]:
banksim_filtrado.dtypes

* *Se obtuvo la conversión de tipo de datos esperada.*

In [ ]:
banksim_filtrado.describe()

* *De los estadísticos expuestos, se destaca el **valor máximo** de la variable `amount`, el cual dista notablemente del valor medio de dicho feature.*

In [ ]:
sns.pairplot(data=banksim_filtrado, hue= 'fraud', vars=['step','age','gender','merchant','category','amount','fraud'])

### 3. PCA (Principal Component Analysis)

#### 3.1 Aplicación del Modelo de Reducción de Dimensionalidad - PCA

* *Se implementará la **técnica de PCA**, ya que es particularmente útil en el tratamiento de datos donde existen múltiples colinealidades entre las características / variables, como ocurre en el presente estudio.*
* *El análisis de componentes principales es una técnica matemática utilizada para la reducción de dimensionalidad. Su objetivo es reducir el número de features, conservando la mayor parte de la información original.*
* *El **propósito principal** de crear un PCA antes de aplicar cualquier técnica de clasificación, será visualizar en 2D cómo se agrupan las transacciones de fraude y no fraude y si existe una clara separación entre ellas.*
* *Así, la aplicación de PCA, se llevará a cabo sin previa aplicación de técnica de sobremuestreo, principalmente para que la reducción de la dimensionalidad, se realice en los datos de entrenamiento principales (orginales) y se logre obtener una real visualización de los datos.*

* *Como **X** vamos a considerar 6 variables -que serán escaladas- que sirven como predictoras, con el fin de luego reducir su dimensionalidad y como **y** al feature `fraud`.*
* *Como se expuso anteriormente, algunas de ellas parece tener fuerte influencia en la determinación si una operación es fraudulente o no (como el gasto y el tipo de categoría), y otras no (como los pasos o el género).*

1. Se **seleccionan las variables** predictoras (`X`) y la variable a predecir (`y`).

In [ ]:
X = banksim_filtrado[['step','age','gender','merchant','category','amount']]
y = banksim_filtrado[['fraud']]

2. Se **escalan los datos**. Normalizamos las X.

In [ ]:
from sklearn.preprocessing import RobustScaler # Escala características usando estadísticas que sean robustas a valores atípicos

X_escalado = RobustScaler().fit_transform(X)
X_escalado[:6]

1. Se define la **matriz de Covarianza**.

In [ ]:
# Matriz de covarianza
features = X_escalado.T
cov_matrix = np.cov(features)
cov_matrix[:6]

* En la diagonal de la matriz de covarianzas, tenemos varianzas, y los demás elementos son las covarianzas.

3. Se realiza la **Eigendecomposition**.

In [ ]:
# Eigendecomposition
valores, vectores = np.linalg.eig (cov_matrix) 

* *A partir de ésto, podemos calcular el porcentaje de varianza explicada (explained variance) por componente principal:*

In [ ]:
varianzas_explicadas = [] 
for i in range (len (valores)): 
    varianzas_explicadas.append (valores [i] / np.sum (valores)) 
 
    print (np.sum (varianzas_explicadas), '\n', varianzas_explicadas)

* *El primer valor (penúltima fila) es solo la suma de las varianzas explicadas y debe ser igual a 1. El segundo valor (última fila) es una matriz, que representa el porcentaje de varianza explicada por componente principal.*
* *El primer componente principal representa el 62% de la varianza de los datos, el segundo el 32%.*

* 4. **Visualizaciones**.

In [ ]:
proyectado_1 = X_escalado.dot (vectores.T [0]) 
proyectado_2 = X_escalado.dot (vectores.T [1])
res = pd.DataFrame (proyectado_1, columns = ['PC1']) 
res ['PC2'] = proyectado_2 
res ['Y'] = y

* *Primero se visualiza el conjunto de datos en una dimensión: como una línea (no incluímos a PC2).*

In [ ]:
plt.figure(figsize=(10, 5))
sns.scatterplot (res ['PC1'], [0] * len (res), hue = res ['Y'], s = 200)

* *Luego, se exponen los datos en un espacio 2D:*

In [ ]:
plt.figure (figsize = (10, 5)) 
sns.scatterplot(res['PC1'], res['PC2'], hue=res['Y'], s=100)

* *En ambos gráficos se observa que las variables parecen relativamente fácil de separar.*
* *Casi la totalidad de las transacciones No Fraudulentas están concentradas en el PC2, mientras que las transacciones fraudulentas, se encuentran más extendidas en la dimensionalidad del PC1.*

5. Vemos cómo funciona el **modelo PCA**, con 2 componentes principales.

In [ ]:
from sklearn.decomposition import PCA # Para poder realizar la reducción de dimensionalidad

pca = PCA(n_components=2)
X_pca = pca.fit_transform(X_escalado)
print("original shape:   ", X.shape)
print("transformed shape:", X_pca.shape)

In [ ]:
print(pca.components_)

* *Los datos transformados se han reducido a una sola dimensión.*
* *La reducción de dimensionalidad de PCA, eliminó la información a lo largo del eje o ejes principales menos importantes, dejando solo el componente o componentes de los datos con la mayor varianza, en éste caso, el primer y segundo componente.*
* *Éste conjunto de datos de dimensión reducida es, en algunos sentidos, "lo suficientemente bueno" para codificar las relaciones más importantes entre los puntos: a pesar de reducir la dimensión de los datos, la relación general entre los puntos de datos se conserva en su mayoría.*

#### 3.2 Entrenamiento del modelo de Árbol de Decisión, una vez realizada la Reducción de Dimensionalidad

In [ ]:
from sklearn.model_selection import train_test_split # Para dividir los datos en subconjuntos de entrenamiento y prueba

X_train, X_test, y_train, y_test = train_test_split(X_pca, y, test_size=0.30, random_state=10)

In [ ]:
from sklearn.tree import DecisionTreeRegressor # Regresor deL Árbol de Decisión
from sklearn.metrics import mean_squared_error # Para cálculo del RMSE

lista_rmse_train_dt = []
lista_rmse_test_dt = []

max_depths = [1,2,3,4,5,6,7,8,9,10,20,25,30,40,50,80]

for max_depth in max_depths:

    # Se define el modelo con la profundidad deseada
    tree_regressor = DecisionTreeRegressor(max_depth = max_depth, random_state=10)
    
    tree_regressor.fit(X_train, y_train)
    
    y_train_pred = tree_regressor.predict(X_train)
    rmse_train = np.sqrt(mean_squared_error(y_train, y_train_pred))
    
    y_test_pred = tree_regressor.predict(X_test)
    rmse_test = np.sqrt(mean_squared_error(y_test, y_test_pred))
    
    lista_rmse_train_dt.append(rmse_train)
    lista_rmse_test_dt.append(rmse_test)

In [ ]:
plt.figure(figsize = (8,4))
plt.plot(max_depths, lista_rmse_train_dt,'o-',label='train' )
plt.plot(max_depths, lista_rmse_test_dt,'o-',label='test')
plt.legend()
plt.xlabel("Max. Profundidad")
plt.ylabel("RMSE")
plt.title('Curva de Validación - Árbol de Decisión')

In [ ]:
regresor = DecisionTreeRegressor(max_depth=10, random_state=42)
regresor.fit(X_train,y_train)

In [ ]:
y_train_pred = regresor.predict(X_train)
y_test_pred = regresor.predict(X_test)

In [ ]:
rmse_train = np.sqrt(mean_squared_error(y_train, y_train_pred))
rmse_test = np.sqrt(mean_squared_error(y_test, y_test_pred))
print(f'Raíz del error cuadrático medio en Train: {rmse_train}')
print(f'Raíz del error cuadrático medio en Test: {rmse_test}')

* *Los RMSE tanto para Train como para Test, resultan similares entre sí, y a su vez con buen desempeño.*

In [ ]:
regresor.feature_importances_

* *La primer característica tiene una mayor importancia relativa a la hora de realizar predicciones (su importancia es del 75%).*
* *El mismo posee una buena capacidad de predicción, por el hecho de haber realizado reducción de dimensionalidad, pasando de 6 features a 2 estimadores.*
* *No se observa en éste caso sobrejuste, ya que los RMSE obtenidos para Train y Test son similares.*

### 4. Sobremuestreo (Oversampling) con SMOTE

* *En el presente análisis, se llevará a cabo una técnica de sobremuestreo llamada SMOTE [3] (técnica de sobremuestreo de minorías sintéticas).* 
* *SMOTE creará nuevos puntos de datos de la clase minoritaria utilizando las instancias vecinas, de modo que las muestras generadas no son copias exactas, sino que son similares a las instancias que tenemos.*

1. Se **seleccionan las variables** predictoras (`X`) y la variable a predecir (`y`).

In [ ]:
X = banksim_filtrado[['step','age','gender','merchant','category','amount']]
y = banksim_filtrado[['fraud']]

print(X.head(),"\n")
print(y.head())

In [ ]:
y[y==1].count()

* *Vemos el desbalanceo de los datos.*

2. Aplicamos **SMOTE**.

In [ ]:
from imblearn.over_sampling import SMOTE # Importamos librería SMOTE, para poder realizar el sobremuestreo de los datos

#Transformamos el Dataset
sm = SMOTE(random_state=42)
X_res, y_res = sm.fit_resample(X, y)
y_res = pd.DataFrame(y_res)
print(y_res.value_counts())

* *Una vez aplicado SMOTE para equilibrar el conjunto de datos, los resultados muestran que tenemos el número exacto de instancias de clase (1 y 0)*

### 5. Modelos de Machine Learning

* *__Aclaración:__ Los clasificadores basados en Árboles de decisión y los métodos de conjuntos basados en Árboles (RF, XGB) son invariantes al escalado de características (no lo requieren para converger a los errores mínimos), pero aún así, podría ser una buena idea reescalar / estandarizar su datos.*  
* *En el presente apartado, se optó por llevar a cabo el modelo de Random Forest, **sin previo escalado de los datos**.*

1. Se realiza el __Train Test Split__ para dividir el entrenamiento del test.

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_res,y_res,test_size=0.3,random_state=42,shuffle=True,stratify=y_res)

2. Se define una función para trazar la __curva ROC_AUC__.

* *La curva ROC-AUC, es una buena forma visual de ver el performance de los modelos de clasificación a confeccionar.*

In [ ]:
# Función para trazar la curva ROC_AUC

def plot_roc_auc(y_test, preds):
    fpr, tpr, threshold = roc_curve(y_test, preds)
    roc_auc = auc(fpr, tpr)
    plt.title('ROC curve')
    plt.plot(fpr, tpr, 'b', label = 'AUC = %0.2f' % roc_auc)
    plt.legend(loc = 'lower right')
    plt.plot([0, 1], [0, 1],'r--')
    plt.xlim([0, 1])
    plt.ylim([0, 1])
    plt.ylabel('Ratio Verdadero Positivo')
    plt.xlabel('Ratio Falso Positivo')
    plt.show()

#### 5.1 Modelo Benchmark - Árbol de Decisión

In [ ]:
from sklearn.tree import DecisionTreeClassifier # Clasificador deL Árbol de Decisión
from sklearn.metrics import classification_report # Crea un informe de texto que muestra las principales métricas de clasificación
from sklearn.metrics import confusion_matrix # Calcula la matriz de confusión para evaluar la precisión de una clasificación
from sklearn.metrics import roc_curve, auc # Para visualizar la Curva ROC y el AUC
from sklearn.metrics import accuracy_score # Calcula el accuracy de una clasificación (incluído en classification_report)

tree = DecisionTreeClassifier(max_depth=4, random_state=10)

tree.fit(X_train,y_train)
y_pred = tree.predict(X_test)


print("Reporte de Clasificación para Árbol de Decisión: \n", classification_report(y_test, y_pred))
print("Matriz de Confusión para Árbol de Decisión: \n", confusion_matrix(y_test,y_pred))
plot_roc_auc(y_test, tree.predict_proba(X_test)[:,1])

* *Vemos que un modelo simple de Árboles de Decisión, parece tener un buen desempeño, determinado principalmente en los valores obtenidos de las métricas `precision` y `accuracy`*
* *El mayor error en el que incurre el modelo, es en los `falsos negativos`, es decir en aquellas transacciones No Fraudulentas, que fueron predichas como Fraudulentas.*

In [ ]:
importances = tree.feature_importances_
columns = X.columns
sns.barplot(columns, importances)
plt.title('Importancia de cada Feature')
plt.show()

* *Como se observa, la variable más importante a la hora de predecir, es `amount` por lejos.*

#### 5.2. Random Forest Classifier

* *Se elije el modelo de ML Random Forest, por ser **muy popular** para la detección de fraudes.*

* *__Random Forest__ es un tipo de algoritmo de aprendizaje automático supervisado basado en el aprendizaje por conjuntos.*
* *El aprendizaje conjunto es un tipo de aprendizaje en el que se combinan diferentes tipos de algoritmos o el mismo algoritmo varias veces para formar un modelo de predicción más potente.*
* *El algoritmo de bosque aleatorio combina varios algoritmos del mismo tipo, es decir, varios árboles de decisión , lo que da como resultado un bosque de árboles , de ahí el nombre "Random Forest" o Bosque Aleatorio.*
* *Dicho algoritmo, al igual que el Árbol de Decisión, es un **modelo de aprendizaje supervisado para clasificación**, aunque también puede usarse para problemas de regresión.*
* *No debe dejar de nombrarse su principal desventaja: debido a su complejidad, requieren mucho más tiempo para entrenarse que otros algoritmos comparables.*

##### 5.2.1 Optimización de Hiperparámetros: RandomSearch

* *El ajuste de los hiperparámetros puede ser ventajoso para crear un modelo que sea mejor en la clasificación.*
* *En el caso de Random Forest, puede no ser necesario, ya que los Bosques Aleatorios ya son muy buenos en la clasificación.*
* *Utilizar una búsqueda exhaustiva de GridSearch para elegir los valores de los hiperparámetros también puede llevar mucho tiempo.*
* *Sin embargo, en los casos en que sólo hay unos pocos valores potenciales para sus hiperparámetros o cuando su modelo de clasificación inicial no es muy preciso, podría ser una buena idea investigar al menos el efecto de cambiar algunos de los valores de hiperparámetros en su modelo.*

* *En el presente estudio, en el que el Dataset con el que trabajamos tiene un gran número de instancias, la aplicación de Gridsearch podría demorar horas, y el resultado de aplicarlo o no sería similar.*
* *Es por ello, que **a modo de demostración**, es decir, que no es realmente necesario de aplicar, se realizará una optimización de hiperparámetros a partir de RandomSearch, que también demora (unos 30 minutos aproximadamente), pero es más viable en función de los tiempos.*

* *Se puede optar por no correr la siguiente celda, ya que, los mejores hiperparámetros devueltos a partir de RandomSearch, fueron considerados en el punto `5.2.2`.*

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score,KFold
from sklearn.model_selection import RandomizedSearchCV

rf_clf_rs = RandomForestClassifier(random_state=42)
n_estimators = [50,100,150,200]
max_features = ['auto', 'sqrt']
max_depth = [2,4,6,8]

param_grid = dict(max_depth=max_depth,max_features=max_features,n_estimators=n_estimators)
kfold = KFold(n_splits=2, shuffle=True, random_state=0)
ramdom = RandomizedSearchCV(estimator=rf_clf_rs,param_distributions=param_grid,verbose=1,n_iter=32,cv=kfold,n_jobs=-1)
# n_iter=32 --> The total space of parameters is 32 --> máximo de iteraciones que me permite realizar

ramdom_result = ramdom.fit(X_train, y_train.values.ravel())

In [ ]:
print("Mejores hiperparametros: "+str(ramdom_result.best_params_))
print("Mejor Score: "+str(ramdom_result.best_score_)+'\n')

##### 5.2.2 Se entrena el modelo Random Forest con los argumentos obtenidos de RandomSearch

In [ ]:
import sklearn.metrics as metrics

rf_clf_rs = RandomForestClassifier(n_estimators=50,max_features='auto',max_depth=8,random_state=42,
                                verbose=1,class_weight="balanced",oob_score=True,n_jobs=-1)

rf_clf_rs.fit(X_train,y_train.values.ravel())
y_test_pred = rf_clf_rs.predict(X_test)

print("Reporte de Clasificación para Random Forest Classifier: \n", classification_report(y_test, y_test_pred))
print("Matriz de Confusión para Random Forest Classifier: \n", confusion_matrix(y_test,y_test_pred))
plot_roc_auc(y_test, rf_clf_rs.predict_proba(X_test)[:,1])

* *El desempeño del presente modelo, ha demostrado una notable mejoría en relación al de Árboles de Decisión.*
* *El hecho de que en la curva ROC-AUC, el **AUC sea = 1**, expone que el valor diagnóstico es perfecto.* 

In [ ]:
y_test_pred = rf_clf_rs.predict(X_test)
y_train_pred = rf_clf_rs.predict(X_train)

In [ ]:
print('Accuracy sobre el train set: ', metrics.accuracy_score(y_train, y_train_pred)) # Accuracy Asociado al Train
print('Accuracy sobre el test set: ', metrics.accuracy_score(y_test, y_test_pred)) # Accuracy Asociado al Test

* *Además, tenemos en éste caso, **bajo sesgo** y **baja varianza**, ya que el desempeño es bueno tanto en el train como en el test.*
* *Ésto es precisamente lo que se busca en un modelo, por lo que el presente escenario es muy favorable, ya que la complejidad resulta ser la justa y necesaria.*

In [ ]:
importances = rf_clf_rs.feature_importances_
columns = X.columns
sns.barplot(columns, importances)
plt.title('Importancia de cada Feature')
plt.show()

* *La variable más importante a la hora de predecir sigue siendo `amount`, aunque ahora adquieren notable relevancia las atributos de `category` y `merchant`.*

### 6. Comparación del Desempeño de los Modelos de ML Desarrollados

|      Modelos      | Precision No Fraude | Precisión Fraude |            Hiperparámetros Utilizados           |
|:-----------------:|:-------------------:|:----------------:|:-----------------------------------------------:|
| Árbol de Decisión |         0.98        |       0.95       |         max_features=none,max_depth=4           |
|   Random Forest   |         0.99        |       0.97       | n_estimators=50,max_features='auto',max_depth=8 |

* *En éste caso, fue **Random Forest fue el modelo con mejor desempeño**.*
* *Éste resultado, era realmente el esperado, ya que se trata de un algoritmo eficiente y fácil de usar que ofrece un alto rendimiento y precisión en comparación con otros algoritmos.*
* *Random Forest, además de aplicar Bagging, también selecciona features al azar, de esa manera descorrelaciona aún más los distintos modelos de árbol creados.*
* *Cuenta con una **doble aleatoriedad**: tanto en la selección del valor k de características para cada árbol como en la cantidad de muestras que usa para entrenar cada árbol creado.*
* *El algoritmo de bosque aleatorio **no está sesgado** (lo cual coincide con lo observado anteriormente), ya que hay varios árboles y cada árbol se entrena en un subconjunto de datos. Básicamente, el algoritmo de bosque aleatorio se basa en el poder de "la multitud"; por lo tanto, se reduce el sesgo general del algoritmo.*

__¿Cuál es el beneficio de usar Random Forest versus Árboles de Decisión?__
* *Los bosques aleatorios son una opción más robusta que un simple árbol de decisión.*
* *Como Random Forest consiste en una colección de árboles en un subconjunto aleatorio de características, tiene como predicciones finales, los resultados combinados de esos árboles.*
* *Éste algoritmo **es muy estable**. Incluso si se introduce un nuevo punto de datos en el conjunto de datos, el algoritmo general no se ve muy afectado, ya que los nuevos datos pueden afectar a un árbol, pero es muy difícil que afecte a todos los árboles.*
* *Random Forest **previene el sobreajuste la mayoría de las veces**, creando subconjuntos aleatorios de las características y construyendo árboles más pequeños usando estos subconjuntos. Después, combina los subárboles de submuestras de características, por lo que no tiende a sobreajustarse a todo su conjunto de características de la manera en que lo hacen los Árboles de Decisión.*

### 7. Conclusión

* En el presente estudio se buscó realizar la Detección de Fraudes en los datos de pago de nuestro banco elegido, denominado Banksim.
* Como el conjunto de datos de Fraude del Dataset utilizado, tiene un gran problema de desequilibrio, se realizó una técnica de sobremuestreo llamada SMOTE para generar nuevos puntos de datos de la clase minoritaria y así equilibrar las instancias con transacciones fraudulentas y no fraudulentas.
* Para la Detección, se llevaron a cabo distintos modelos de ML, habiendo alcanzado muy buenos resultados con los clasificadores elegidos, entre otras razones, por haberse elegido entre ellos el modelo de Random Forest, el cual es muy popular y efectivo a la hora de detectar Fraudes Bancarios.
* La razón por la cual elegí trabajar con 'Fraude Bancario' para mi Trabajo Final, es que por un lado, soy economista, trabajo en un banco en el sector de Riesgo Crediticio y considero muy enriquecedor lograr detectar de forma rápida y precisa, la existencia de fraudes, para evaluar luego si es necesario hacer un mayor seguimiento a ciertos tipos de transacciones, o bloquear las mismas. Por otro lado, es sorprendente la cantidad de ingresos que pierden todo tipo de empresas debido a ello, y es realmente atrapante intentar al menos, poder participar para mejorar dicha situación a futuro.

### 8. Recursos

[1]. Encuesta Global de Fraude Bancario 2019 - KPMG (Mayo 2019). Obtenido de https://home.kpmg/cr/es/home/tendencias/2019/07/fraude_bancario.html  
[2]. Synthetic data from a financial payment system. Obtenido de https://www.kaggle.com/ntnu-testimon/banksim1  
[3]. SMOTE: Synthetic Minority Over-sampling Technique. Obtenido de https://machinelearningmastery.com/smote-oversampling-for-imbalanced-classification/